In [ ]:
import keras
# import the class
from functions_final import DeepQLearning
import pandas as pd
# numpy
import numpy as np
import random

In [ ]:
TEST_PATH='../data/matrix1/normal/test_csic2010-normal.csv'
# TEST_PATH='../data/matrix1/normal/test_fwaf-normal.csv'
# TEST_PATH='../data/matrix1/normal/test_http_param-normal.csv'

dataset = pd.read_csv(TEST_PATH)
# print(type(dataset))

In [ ]:
# select the parameters
gamma=1
# probability parameter for the epsilon-greedy approach
epsilon=0.1
# number of training episodes
# NOTE HERE THAT AFTER CERTAIN NUMBERS OF EPISODES, WHEN THE PARAMTERS ARE LEARNED
# THE EPISODE WILL BE LONG, AT THAT POINT YOU CAN STOP THE TRAINING PROCESS BY PRESSING CTRL+C
# DO NOT WORRY, THE PARAMETERS WILL BE MEMORIZED
# numberEpisodes=100

state_size = 38  # column in dataset exculde label
action_size = 2
batch_size = 100 # mô hình cập nhật sau khi train 100 dữ liệu
# episodes = data.shape[0]

In [ ]:
# load the model
loaded_model = keras.models.load_model("trained_model.h5",custom_objects={'my_loss_fn':DeepQLearning.my_loss_fn})

sumObtainedRewards=0

In [ ]:
def step(action, index):
    if action == dataset.iloc[index].values[-1]: 
        reward = 1
    else:  
        reward = -1
    # print ("reward",reward)
    next_state = dataset.iloc[index + 1].values[:-1]
    next_state = np.reshape(next_state, [1,state_size])

    # print(dataset.iloc[index].values[:-1])
    # print("nextState")
    # print(dataset.iloc[index + 1].values[:-1])
    
    if index == (dataset.shape[0]-1):
        terminated = True
    terminated = False
    state = next_state
    # print("_____________")
    return np.array(state, dtype=np.float32), reward, terminated, False, {}


In [ ]:
# create the environment, here you need to keep render_mode='rgb_array' since otherwise it will not generate the movie
# reset the environment
for row in range(0,len(dataset),1):

    # (currentState,prob)=env.reset()
    currentState=random.randint(0, len(dataset)-1)
    # since the initial state is not a terminal state, set this flag to false
    terminalState=False
    while not terminalState:
        # get the Q-value (1 by 2 vector)
        Qvalues=loaded_model.predict(currentState.reshape(1,state_size))
        # select the action that gives the max Qvalue
        action=np.random.choice(np.where(Qvalues[0,:]==np.max(Qvalues[0,:]))[0])
        # if you want random actions for comparison
        #action = env.action_space.sample()
        # apply the action
        (currentState, currentReward, terminalState,_,_) = step(action, row)
        # sum the rewards
        sumObtainedRewards+=currentReward
